In [2]:
import csv
import pandas as pd
import numpy as np
import glob

sagrada_clean_dataset = pd.read_csv(r'clean_dataset\sagrada_clean_dataset.csv', encoding='utf-8',index_col=0)
mila_clean_dataset = pd.read_csv(r'clean_dataset\mila_clean_dataset.csv', encoding='utf-8',index_col=0)
parkguell_clean_dataset = pd.read_csv(r'clean_dataset\parkguell_clean_dataset.csv', encoding='utf-8',index_col=0)
batllo_clean_dataset = pd.read_csv(r'clean_dataset\batllo_clean_dataset.csv', encoding='utf-8',index_col=0)
bcn_clean_dataset = pd.read_csv(r'clean_dataset\bcndata_clean_dataset.csv', encoding='utf-8',index_col=0)

alldata = pd.concat((sagrada_clean_dataset, mila_clean_dataset, parkguell_clean_dataset, batllo_clean_dataset,bcn_clean_dataset) ,ignore_index=True)

In [3]:
def is_official(user):
    official_words = ['导游','旅行社','旅行','猫途鹰','巴特罗之家','摄影','酒店','房地产','国家展览','资讯','地接','包车','接待','代购','攻略','旅游网','加泰罗尼亚旅游官网','旅拍','有限公司']
    for official_word in official_words:
        if official_word in str(user):
            return True
    return False

In [4]:
alldata['ifoficial'] = ['yes' if is_official(user) else None for user in alldata['USER']]
image_ofi = alldata.dropna(subset = ['ifoficial'])
no_oficial = alldata[alldata['ifoficial'] != 'yes']
no_oficial['iflocation'] = ['yes' if '西班牙' in str(x) else None for x in no_oficial['LOCATION']]
image_perci = no_oficial.dropna(subset = ['iflocation'])
no_oficial_no_location = no_oficial[no_oficial['iflocation'] != 'yes']
image_expect = no_oficial_no_location

<ipython-input-4-44a049064391>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_oficial['iflocation'] = ['yes' if '西班牙' in str(x) else None for x in no_oficial['LOCATION']]


In [ ]:
from bixin import predict
image_perci['score'] = image_perci['CONTENT'].astype(str).apply(predict)
posi_perci = image_perci[image_perci['score'] > 0]
nega_perci = image_perci[image_perci['score'] < 0]

In [ ]:
lda_topic_modeling(text = posi_perci["content_cutted"],max_df_lda=0.1,min_df_lda=5, topics_number = 3, max_iter = 50)

In [ ]:
lda_topic_modeling(text = nega_perci["content_cutted"],max_df_lda=0.2,min_df_lda=3, topics_number = 5, max_iter = 50)

In [ ]:
%run sentiment_analysis_mydata.ipynb

In [ ]:
#text1=image_perci.iloc[1,1]
#sentiment_analysis(text1)
image_perci['score'] = image_perci['CONTENT'].astype(str).apply(sentiment_analysis_mydata)
image_perci
#s.to_csv('test_sentiment_analysis.csv', encoding='utf-8')

In [ ]:
image_perci['score'] = image_perci['score'].astype(str)
nega_perci = image_perci[image_perci['score'] == '[0]']
nega_perci

In [ ]:
nega_perci.to_csv('test_sentiment_analysis.csv', encoding='utf-8')

In [ ]:
nega_perci["content_cutted"] = nega_perci['CONTENT'].apply(chinese_word_segmentation)
lda_topic_modeling(text = nega_perci["content_cutted"],max_df_lda=0.2,min_df_lda=3, topics_number = 5, max_iter = 50)